# Importing Data and Libraries

In [138]:
%pylab inline
import nltk
nltk.download('wordnet')
import pandas as pd
from gensim.models import LdaModel
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package wordnet to /Users/kaushik-
[nltk_data]     shakkari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [123]:
df =pd.read_csv('ICD_preprocessed_dataset',index_col=[0])

In [124]:
df.head()

,Record #,Chief Complaint,Age,Sex,Acuity (initial),Area of Care,Disposition,ICD-9 1,In ED,ICD-level1,class
0,1,nausea vomiting hx ca,38yr,M,3.0,West,Home,787.01,47.0,787,SYMPTOMS
1,2,earache,57yr,F,5.0,UADC,Home,388.70,60.0,388,DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS
2,3,right foot pain,53yr,M,4.0,West,Home,V67.59,146.0,V,SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUE...
3,4,danger others,26yr,F,4.0,Psych Lock-down,Transfer to Outside Facility,298.9,363.0,298,MENTAL DISORDERS
4,5,leg laceration,32yr,M,3.0,West,Home,890.0,208.0,890,INJURY AND POISONING


In [129]:
docs = df['Chief Complaint'].astype(str)

In [130]:
docs = [doc.split() for doc in docs]

# Generating Bigram and Trigram# 

In [131]:
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
           
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            
            docs[idx].append(token)

/Users/kaushik-shakkari/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [132]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 3852
Number of unique words after removing rare and common words: 884


In [133]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [134]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 884
Number of documents: 32112


# Topic Modelling using LDA

In [135]:
num_topics = 30
chunksize = 500 
passes = 20 
iterations = 400
eval_every = 1  


temp = dictionary[0] 
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

/Users/kaushik-shakkari/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 2min 19s, sys: 568 ms, total: 2min 20s
Wall time: 2min 20s


# Visualizing Topic Model

In [136]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [137]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/Users/kaushik-shakkari/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
20     0.035280 -0.005477       1        1  6.702743
14    -0.090132  0.059605       2        1  5.878407
18     0.035125 -0.003664       3        1  4.929637
23     0.040669 -0.079965       4        1  4.636889
7      0.035190 -0.001699       5        1  4.311276
4      0.048392 -0.248110       6        1  4.145386
5      0.012228 -0.233972       7        1  4.018662
8     -0.373684  0.015841       8        1  3.926100
16     0.016264  0.001836       9        1  3.793999
24     0.038887 -0.050529      10        1  3.748104
17     0.035192 -0.001958      11        1  3.738097
27     0.035061 -0.002277      12        1  3.478163
15    -0.153358  0.065538      13        1  3.233523
6      0.035037 -0.002054      14        1  3.208168
10     0.035107 -0.002988      15        1  3.167425
9      0.035119 -0.001968      16        1  3.163801
26     0.035024 -0.002269      17        1  3.083454
2      0.035042 -0.003733      18        1  2.968852
29     0.034947 -0.002012      19        1  2.833527
11     0.034944 -0.001838      20        1  2.810615
3     -0.329793 -0.034726      21        1  2.788675
22     0.051239 -0.014510      22        1  2.619880
12     0.051254 -0.014459      23        1  2.619630
28     0.037141 -0.031035      24        1  2.480794
1      0.061279  0.258956      25        1  2.378289
21     0.046042  0.125306      26        1  2.273260
19     0.034789  0.000165      27        1  2.158374
0      0.034807 -0.000638      28        1  1.905258
13    -0.001290 -0.010050      29        1  1.782189
25     0.054197  0.222684      30        1  1.216822, topic_info=    Category         Freq                Term        Total  loglift  logprob
33   Default  2349.000000           abdominal  2349.000000  30.0000  30.0000
91   Default  2236.000000   medication_refill  2236.000000  29.0000  29.0000
43   Default  1711.000000               chest  1711.000000  28.0000  28.0000
49   Default  1648.000000                left  1648.000000  27.0000  27.0000
7    Default  1507.000000               right  1507.000000  26.0000  26.0000
116  Default  1604.000000    vaginal_bleeding  1604.000000  25.0000  25.0000
150  Default  1573.000000                   x  1573.000000  24.0000  24.0000
75   Default  1339.000000                back  1339.000000  23.0000  23.0000
32   Default  1094.000000  suicidal_ideations  1094.000000  22.0000  22.0000
57   Default  1249.000000              injury  1249.000000  21.0000  21.0000
12   Default  1041.000000                 leg  1041.000000  20.0000  20.0000
159  Default   795.000000                knee   795.000000  19.0000  19.0000
79   Default   790.000000         sore_throat   790.000000  18.0000  18.0000
39   Default   792.000000                 sob   792.000000  17.0000  17.0000
169  Default   740.000000            swelling   740.000000  16.0000  16.0000
223  Default   809.000000                   r   809.000000  15.0000  15.0000
148  Default   747.000000                 abd   747.000000  14.0000  14.0000
74   Default   671.000000          psych_exam   671.000000  13.0000  13.0000
92   Default   872.000000              refill   872.000000  12.0000  12.0000
61   Default   597.000000           headaches   597.000000  11.0000  11.0000
110  Default   745.000000                hand   745.000000  10.0000  10.0000
6    Default   687.000000                foot   687.000000   9.0000   9.0000
90   Default   788.000000          medication   788.000000   8.0000   8.0000
112  Default   749.000000            bleeding   749.000000   7.0000   7.0000
24   Default   696.000000                 eye   696.000000   6.0000   6.0000
111  Default   603.000000           dizziness   603.000000   5.0000   5.0000
4    Default   577.000000            vomiting   577.000000   4.0000   4.0000
115  Default   726.000000             vaginal   726.000000   3.0000   3.0000
400  Default   628.000000   